# Color-DCT image compression

Removing redundancy in the color domain with the DCT, that is an orthonormal (ortongonal + unitary) and energy preserving transform.

In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.axes as ax
#plt.rcParams['text.usetex'] = True
#plt.rcParams['text.latex.preamble'] = [r'\usepackage{amsmath}'] #for \text command
import pylab
import math
import numpy as np
from scipy import signal
import cv2
import os
!ln -sf ~/quantization/deadzone_quantizer.py .
!ln -sf ~/quantization/midtread_quantizer.py .
!ln -sf ~/quantization/midrise_quantizer.py .
!ln -sf ~/quantization/distortion.py .
!ln -sf ~/quantization/information.py .
!ln -sf ~/MRVC/src/YCrCb.py .
!ln -sf ~/MRVC/src/debug.py .
!ln -sf ~/MRVC/src/image_3.py .
!ln -sf ~/MRVC/src/image_1.py .
!ln -sf ../common.py .
import deadzone_quantizer as deadzone
import midtread_quantizer as midtread
import midrise_quantizer as midrise
import YCrCb
import distortion
import information
import image_3 as RGB_image
import image_1 as gray_image
import colored
import common

## Configuration

In [ ]:
# Prefix of the RGB image to be quantized.

home = os.environ["HOME"]
#fn = home + "/MRVC/sequences/stockholm/"
fn = home + "/MRVC/sequences/lena_color/"

DCT_components = ['0', '1', '2']

# Number of quantization steps.
N_Q_steps = 8
#Q_steps = [256*i/N_Q_steps for i in range(N_Q_steps + 1, 0, -1)]
Q_steps = [2**i for i in range(7, -1, -1)]
print(Q_steps)

#quantizer = midtread
quantizer = deadzone
#quantizer = midrise

## Read the image and show it

In [ ]:
RGB_img = RGB_image.read(fn).astype(np.int16)
#RGB_img -= np.average(RGB_img).astype(np.int16)
common.show(RGB_img, fn + "000.png")

In [ ]:
RGB_img.shape

## (RGB -> ColorDCT) transform of the image

In [ ]:
import numpy as np
from scipy.fftpack import dct, idct

DCT_type = 3
norm = "ortho" # Orthonormal: orthogonal + unitary (unit gain in both directions of the transform)
#norm = None

In [ ]:
def color_DCT(RGB_img):
    DCT_img = np.empty_like(RGB_img)
    for y in range(RGB_img.shape[0]):
        for x in range(RGB_img.shape[1]):
            DCT_img[y, x] = dct(RGB_img[y, x], type=DCT_type, norm=norm)
    return DCT_img

def color_IDCT(DCT_img):
    RGB_img = np.empty_like(DCT_img)
    for y in range(DCT_img.shape[0]):
        for x in range(DCT_img.shape[1]):
            RGB_img[y, x] = idct(DCT_img[y, x], type=DCT_type, norm=norm)
    return RGB_img

In [ ]:
DCT_img = color_DCT(RGB_img)

In [ ]:
common.show_gray(DCT_img[..., 0], fn + "000 (DCT0)")

In [ ]:
common.show_gray(DCT_img[..., 1], fn + "000 (DCT1)")

In [ ]:
common.show_gray(DCT_img[..., 2], fn + "000 (DCT2)")

## Energy of the DCT components

In [ ]:
DCT0_avg_energy = information.average_energy(DCT_img[..., 0])
DCT1_avg_energy = information.average_energy(DCT_img[..., 1])
DCT2_avg_energy = information.average_energy(DCT_img[..., 2])
print(f"Average energy DCT[..., 0] = {DCT0_avg_energy}")
print(f"Average energy DCT[..., 1] = {DCT1_avg_energy}")
print(f"Average energy DCT[..., 2] = {DCT2_avg_energy}")
total_DCT_avg_energy = DCT0_avg_energy + DCT1_avg_energy + DCT2_avg_energy
print(f"Total average energy (computed by adding the energies of the DCT coefficients) = {total_DCT_avg_energy}")
print(f"Total RGB average energy (computed directly from the RGB image) = {information.average_energy(RGB_img)*3}")

Therefore, the forward DCT is almost energy preserving.

In [ ]:
RGB_recons_img = color_IDCT(DCT_img)
print(f"Total RGB average energy (computed from the reconstructed RGB image) = {information.average_energy(RGB_recons_img)*3}")

And the same can be said of the backward transform.

## (RGB <-> ColorDCT) transform error

In [ ]:
common.show(RGB_recons_img, fn + "000.png (DCT recons)")

In [ ]:
#RGB_img2 = YCrCb.to_RGB(DCT_img)

In [ ]:
np.array_equal(RGB_img, RGB_recons_img)

In [ ]:
print(RGB_img.max(), RGB_img.min())

In [ ]:
print(RGB_recons_img.max(), RGB_recons_img.min())

In [ ]:
common.show(RGB_img-RGB_recons_img, "Reconstruction error (rounding error) color_DCT")

The DCT transform is irreversible. In general, only integer arithmetic operations guarantees reversibility.

## Synthesis filters gains

The synthesis filters gains are important because the quantization steps of each DCT component should be adjusted in order to effectively provide the desired number of [bins](http://www.winlab.rutgers.edu/~crose/322_html/quantization.pdf) (different dequantized values) in each component.

In [ ]:
def print_info(val):
    DCT0_delta = np.array([val, 0, 0])
    RGB_DCT0_delta = idct(DCT0_delta, type=DCT_type, norm=norm)
    RGB_energy_DCT0_delta = information.energy(RGB_DCT0_delta)
    
    DCT1_delta = np.array([0, val, 0])
    RGB_DCT1_delta = idct(DCT1_delta, type=DCT_type, norm=norm)
    RGB_energy_DCT1_delta = information.energy(RGB_DCT1_delta)
    
    DCT2_delta = np.array([0, 0, val])
    RGB_DCT2_delta = idct(DCT2_delta, type=DCT_type, norm=norm)
    RGB_energy_DCT2_delta = information.energy(RGB_DCT2_delta)
    
    zero = np.array([0, 0, 0])
    RGB_zero = idct(zero, type=DCT_type, norm=norm)
    RGB_energy_zero = information.energy(RGB_zero)
    
    print(f"Energy of {DCT0_delta} in the RGB domain ({RGB_DCT0_delta}) = {RGB_energy_DCT0_delta}")
    print(f"Energy of {DCT1_delta} in the RGB domain ({RGB_DCT1_delta}) = {RGB_energy_DCT1_delta}")
    print(f"Energy of {DCT2_delta} in the RGB domain ({RGB_DCT2_delta}) = {RGB_energy_DCT2_delta}")
    print(f"Energy of {zero} in the RGB domain ({RGB_zero}) = {RGB_energy_zero}")
    
    max_ = max(RGB_energy_DCT0_delta, RGB_energy_DCT1_delta, RGB_energy_DCT2_delta)
    DCT0_relative_gain = RGB_energy_DCT0_delta / max_
    DCT1_relative_gain = RGB_energy_DCT1_delta / max_
    DCT2_relative_gain = RGB_energy_DCT2_delta / max_
    print(f"Relative gain of DCT0 component = {DCT0_relative_gain}")
    print(f"Relative gain of DCT1 component = {DCT1_relative_gain}")
    print(f"Relative gain of DCT2 component = {DCT2_relative_gain}")
    
print_info(255)
print_info(1)
print_info(0)

The gain of each DCT inverse filter is 1. Therefore, the optimal quantization pattern is $\Delta_{\text{DCT0}} = \Delta_{\text{DCT1}} = \Delta_{\text{DCT2}}$.

## Noise in the DCT domain

To decide how to quantize, it is necessary to known how the amplitudes the electronic noise of the input image is transformed to the DCT domain. To see this, we will generate a random image with ([normal](https://numpy.org/doc/stable/reference/random/generated/numpy.random.normal.html)) [Gaussian noise](https://en.wikipedia.org/wiki/Gaussian_noise) with mean 0, DCT-it, and check where the transformed noise has its mean.

In [ ]:
# loc = mean, scale=standard deviation, size=number of samples
RGB_noise = np.random.normal(loc=0, scale=10, size=512*512*3).reshape(512,512,3).astype(np.int8)

In [ ]:
common.show(RGB_noise, "Gaussign RGB noise")

In [ ]:
DCT_noise = dct(RGB_noise, type=DCT_type, norm=norm)

In [ ]:
common.show_gray(DCT_noise[..., 0], "Gaussign DCT0 noise")

In [ ]:
common.show_gray(DCT_noise[..., 1], "Gaussign DCT1 noise")

In [ ]:
common.show_gray(DCT_noise[..., 2], "Gaussign DCT2 noise")

The DCT does not modify the mean of the signal when it has 0 mean. Therefore, the electronic noise (that is white, i.e., have energy in all the amplitudes and frequencies) that produces the smallest [SNR](https://en.wikipedia.org/wiki/Signal-to-noise_ratio) when the amplitude of the input image is also small (less signal, same noise), will be more attenuated by a dead-zone quantizer, when the mean of the signal to quantize is also 0. In other words, the input signal to a dead-zone quantizer must have 0 mean.

## Simple quantization in the DCT domain ($\Delta_{\text{DCT0}} = \Delta_{\text{DCT1}} = \Delta_{\text{DCT2}}$)
Notice that, because the transform is orthogonal, the distortion can be measured in both, the RGB and the DCT domains. In the following cell, for efficiency, the quantizaton error is measured in the transform domain.

In [ ]:
def DCT_same_delta_RD_curve_(RGB_img, Q_steps, quantizer):
    DCT_img = color_DCT(RGB_img)#.astype(np.int16)
    DCT_img[..., 0] -= 128 # Ojo
    points = []
    for Q_step in Q_steps:
        y_DCT, k = quantizer.quan_dequan(DCT_img, Q_step)
        k = (k + 128).astype(np.uint8)
        rate = common.bits_per_color_pixel(k, str(Q_step) + '_')
        #y_DCT[..., 0] += 128                         # Uncomment these lines for measuring
        #y_RGB = inverse_color_IDCT(y_DCT)            # the distortion in the
        #_distortion = distortion.MSE(RGB_img, y_RGB) # RGB domain.
        _distortion = distortion.MSE(DCT_img, y_DCT) # Uncomment this line for measuring the distortion in the DCT domain
        points.append((rate, _distortion))
        print(f"q_step={Q_step:>3}, rate={rate:>7} bits/pixel, distortion={_distortion:>6.1f}")
    return points

def DCT_same_delta_RD_curve(RGB_img, Q_steps, quantizer):
    DCT_img = color_DCT(RGB_img)#.astype(np.int16)
    points = []
    for Q_step in Q_steps:
        y_DCT, k = quantizer.quan_dequan(DCT_img, Q_step)
        k_min = np.min(k)
        #k = (k - np.min(k)).astype(np.uint8)
        rate = common.bits_per_color_pixel((k - k_min).astype(np.uint8), str(Q_step) + '_')
        #y_RGB = inverse_color_IDCT(y_DCT)            # Uncomment these lines for measuring
        #_distortion = distortion.MSE(RGB_img, y_RGB) # the distortion in the RGB domain.
        _distortion = distortion.MSE(DCT_img, y_DCT) # Uncomment this line for measuring the distortion in the DCT domain
        points.append((rate, _distortion))
        print(f"q_step={Q_step:>3}, rate={rate:>7} bits/pixel, distortion={_distortion:>6.1f}")
    return points

DCT_same_delta_RD_points = DCT_same_delta_RD_curve(RGB_img, Q_steps, quantizer)

## Let's see the RD curve

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*DCT_same_delta_RD_points), c='g', marker='.', label="$\Delta_{\mathrm{DCT0}} = \Delta_{\mathrm{DCT1}} = \Delta_{\mathrm{DCT2}}$", linestyle="dashed")
pylab.title("Rate/Distortion")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("MSE in RGB domain")
pylab.legend(loc='upper right')
pylab.show()

## Better RD curves?

The method `DCT_same_delta_RD_curve(img, Q_steps, quantizer)` generates a RD curve where each point is the result of using $\Delta_{\text{DCT0}} = \Delta_{\text{DCT1}} = \Delta_{\text{DCT2}}$. However, a better (at least with more points) RD curve can be generated with:

1. Convert the image from RGB to DCT{0,1,2}.
2. The RD curve of each DCT channel is computed, for a number of quantization steps, measuring the distortion in the RGB domain. We will quantize one component at each iteration and the rest of components will be unquantized. This is necessary to ensure that the low-pass component (DCT0) is always considered in each reconstruction. Otherwise, the distortion (at least using the MSE) will not be estimated correctly.
3. Compute the slope of each segment of the RD curve. Except for the most left point, the slopes are computed as the average between the slopes of the straight lines that connect to the corresponding point.

## RD curve of each DCT componet

In [ ]:
def color_DCT_RD_curve_per_component(RGB_img, Q_steps, Q, components):
    DCT_img = color_DCT(RGB_img)
    DCT_img_copy = DCT_img.copy()
    N_components = len(components)
    RD_points = []
    for c in range(N_components):
        RD_points.append([])
    for Q_step in Q_steps:
        DCT_k = Q.quantize(DCT_img, Q_step)
        for component_index in range(N_components):
            k_min = np.min(DCT_k[..., component_index])
            component_name = components[component_index]
            DCT_y = np.zeros_like(DCT_img)
            DCT_y = DCT_img_copy.copy()
            DCT_y[..., component_index] = Q.dequantize(DCT_k[..., component_index], Q_step)
            #print(Q_step, DCT_k[..., component_index].max(), DCT_k[..., component_index].min(), information.entropy(DCT_k[..., component_index]))
            rate = common.bits_per_gray_pixel(DCT_k[..., component_index] - k_min, str(Q_step) + '_' + component_name + '_' + str(components[component_index]))
            RGB_y = color_IDCT(DCT_y)
            _distortion = distortion.MSE(RGB_img, RGB_y)
            #common.show(RGB_y, components[component_index] + ' ' + str(Q_step))
            RD_points[component_index].append((rate, _distortion, component_name, Q_step))
            print(f"{np.min(DCT_k[..., component_index] - k_min)} {components[component_index]} q_step={Q_step:>3}, rate={rate:>7} bits/pixel, distortion={_distortion:>6.1f}")
    return RD_points
           
DCT_curve_per_component = color_DCT_RD_curve_per_component(RGB_img, Q_steps, quantizer, DCT_components)
DCT_curve_per_component

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*[(i[0], i[1]) for i in DCT_curve_per_component[0]]), c='r', marker='.', label="$\mathrm{DCT0}}$", linestyle="dashed")
pylab.plot(*zip(*[(i[0], i[1]) for i in DCT_curve_per_component[1]]), c='g', marker='.', label="$\mathrm{DCT1}}$", linestyle="dashed")
pylab.plot(*zip(*[(i[0], i[1]) for i in DCT_curve_per_component[2]]), c='b', marker='.', label="$\mathrm{DCT2}}$", linestyle="dashed")
#pylab.plot(*zip(*YCrCb_curve_per_component[0]), c='r', marker='.', label="$\mathrm{Y}}$", linestyle="dashed")
#pylab.plot(*zip(*YCrCb_curve_per_component[1]), c='g', marker='.', label="$\mathrm{Cr}$", linestyle="dashed")
#pylab.plot(*zip(*YCrCb_curve_per_component[2]), c='b', marker='.', label="$\mathrm{Cb}$", linestyle="dashed")
#pylab.plot(*zip(*[(i[0], i[1]) for i in Cr_curve]), c='g', marker='.', label="Cr", linestyle="dashed")
#pylab.plot(*zip(*[(i[0], i[1]) for i in Cb_curve]), c='b', marker='.', label="Cb", linestyle="dashed")
pylab.title("Rate/Distortion")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("MSE in RGB domain")
pylab.legend(loc='upper right')
pylab.show()

## Compute the slope of each point

In [ ]:
def compute_slopes(RD_points):
    counter = 0
    _RD_slopes = []
    points_iterator = iter(RD_points)
    left = next(points_iterator)
    left_BPP = left[0]
    left_MSE = left[1]
    right = next(points_iterator)
    right_BPP = right[0]
    right_MSE = right[1]
    slope = compute_slope(left_BPP, left_MSE, right_BPP, right_MSE)
    _RD_slopes.append((slope, left[2], left[3]))
    _RD_slopes.append((slope, right[2], right[3]))
    left = right
    for right in points_iterator:
        right_BPP = right[0]
        right_MSE = right[1]
        #print(RD_points[counter])
        slope = compute_slope(left_BPP, left_MSE, right_BPP, right_MSE)
        _RD_slopes.append((slope, right[2], right[3]))
        counter += 1
        left = right
        left_BPP = left[0]
        left_MSE = left[1]
        #left_BPP = right_BPP
        #left_MSE = right_MSE
    RD_slopes = []
    slopes_iterator = iter(_RD_slopes)
    left = next(slopes_iterator)
    #print("-->",_RD_slopes)
    #return _RD_slopes
    #RD_slopes.append(left)
    #print(left)
    for right in slopes_iterator:
        #print(left, right)
        RD_slopes.append((((left[0] + right[0])/2), right[1], left[2]))
        left = right
    RD_slopes.append(right)
    return RD_slopes

#print("(Slope (rate, distortion, component, Q_step)) delta_MSE delta_BPP:\n")
DCT0_slopes = compute_slopes(DCT_curve_per_component[0])
DCT1_slopes = compute_slopes(DCT_curve_per_component[1])
DCT2_slopes = compute_slopes(DCT_curve_per_component[2])

In [ ]:
DCT0_slopes

In [ ]:
DCT1_slopes

In [ ]:
DCT2_slopes

## Remove RD point that do not belong to the convex-hull

In [ ]:
def filter_slopes(slopes):
    filtered_slopes = []
    slopes_iterator = iter(slopes)
    prev = next(slopes_iterator)
    for curr in slopes_iterator:
        if prev[0] < curr[0]:
            print(f"deleted {prev}")
        else:
            filtered_slopes.append(prev)
        prev = curr
    filtered_slopes.append(prev)
    return filtered_slopes

In [ ]:
DCT0_slopes = filter_slopes(DCT0_slopes)
DCT0_slopes

In [ ]:
DCT1_slopes = filter_slopes(DCT1_slopes)
DCT1_slopes

In [ ]:
DCT2_slopes = filter_slopes(DCT2_slopes)
DCT2_slopes

## Sort the slopes at each quantization step
Notice that the TPs (Truncation Points) generated in a component must be used in order.

In [ ]:
all_slopes = DCT0_slopes + DCT1_slopes + DCT2_slopes
sorted_slopes = sorted(all_slopes, key=lambda x: x[0])[::-1]
sorted_slopes

## Compute the optimal RD curve
And finally, let's compute the RD curve (remember that the previous points are only an estimation of the order in which the quantization steps should be increased in each component to build the RD curve, not the real RD curve that measures the distortion in the RGB domain).

In [ ]:
def DCT_optimal_curve(RGB_img, sorted_slopes, quantizer, components):
    points = []
    Q_steps_per_component = [256, 256, 256] # This should generate a black image.
    DCT_img = color_DCT(RGB_img)
    #DCT_img[..., 0] -= 128 # Ojo
    k = np.empty_like(DCT_img)
    DCT_y = np.empty_like(DCT_img)
    #YCrCb_img -= 128
    for i in sorted_slopes:
        print(i)
        #point = i[1]
        #component = point[2]
        #Q_step = point[3]
        component, Q_step = i[1], i[2]
        Q_steps_per_component[components.index(component)] = Q_step
        for c, Q_step in zip(components, Q_steps_per_component):
            component_index = components.index(c)
            DCT_y[..., component_index], k[..., component_index] = quantizer.quan_dequan(DCT_img[..., component_index], Q_step)
        k_min = np.min(k[..., component_index])
        k = (k - k_min).astype(np.uint8)
        rate = common.bits_per_color_pixel(k, str(Q_steps_per_component) + '_')
        #y_RGB = YCrCb.to_RGB(y + 128)
        #RGB_y = color_IDCT(DCT_y)                     # Uncomment to compute distortion
        #_distortion = distortion.MSE(RGB_img, RGB_y)  # in the RGB domain.
        _distortion = distortion.MSE(DCT_img, DCT_y)  # Uncomment to compute distortion in the DCT domain.
        #common.show(y_RGB, f"Q_step={Q_steps_per_component}, rate={rate:>7} bits/pixel, distortion={_distortion:>6.1f}")
        points.append((rate, _distortion))
        print(f"Q_step={Q_steps_per_component}, rate={rate:>7} bits/pixel, distortion={_distortion:>6.1f}")
    return points

DCT_optimal_RD_points = DCT_optimal_curve(RGB_img, sorted_slopes, quantizer, DCT_components)
DCT_optimal_RD_points

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*DCT_same_delta_RD_points), c='g', marker='o', label="$\Delta_{\mathrm{DCT0}} = \Delta_{\mathrm{DCT1}} = \Delta_{\mathrm{DCT2}}$", linestyle="dashed")
pylab.plot(*zip(*DCT_optimal_RD_points), c='b', marker='x', label="Sorted Slopes", linestyle="dotted")
pylab.title("Rate/Distortion")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("MSE in RGB domain")
pylab.legend(loc='upper right')
pylab.show()